In [33]:
from crewai.tools import tool
import pandas as pd
import json

import csv
import os
from collections import defaultdict


In [34]:
import csv
import os
from collections import defaultdict

def load_supply_demand_forecast(csv_file="./supply_demand.csv"):
    """
    Loads supply-demand data from a CSV file with columns:
    Year, Quarter, Service, Demand, Supply.

    Returns a nested dictionary formatted as:
    {
      <Service>: {
        "<Year>-<Quarter>": {
          "Demand": <int/float>,
          "Supply": <int/float>
        }
      },
      ...
    }
    """
    forecast_data = defaultdict(dict)

    if not os.path.exists(csv_file):
        print(f"⚠️ Warning: {csv_file} not found!")
        return {}

    with open(csv_file, mode="r", newline="", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        required_cols = ["Year", "Quarter", "Service", "Demand", "Supply"]
        missing_cols = [col for col in required_cols if col not in reader.fieldnames]
        if missing_cols:
            print(f"⚠️ Warning: Missing columns in {csv_file}: {missing_cols}")
            return {}

        for row in reader:
            # Extract the key fields
            year = row["Year"]
            quarter = row["Quarter"]
            service = row["Service"]

            # Convert Demand and Supply to numeric types
            try:
                demand = float(row["Demand"])
                supply = float(row["Supply"])
            except ValueError:
                # If parsing fails, skip or handle differently
                print(f"⚠️ Warning: Non-numeric Demand/Supply in row: {row}")
                continue

            # Construct a sub-key, e.g., "2021-Q1"
            year_quarter_key = f"{year}-{quarter}"

            # Populate the nested dictionary
            forecast_data[service][year_quarter_key] = {
                "Demand": demand,
                "Supply": supply
            }

    return dict(forecast_data)  # Convert defaultdict to a normal dict


In [35]:
forecast=load_supply_demand_forecast()

In [36]:
forecast

{'Cloud Storage': {'2021-Q1': {'Demand': 514.0, 'Supply': 540.0},
  '2021-Q2': {'Demand': 486.0, 'Supply': 494.0},
  '2021-Q3': {'Demand': 524.0, 'Supply': 529.0},
  '2021-Q4': {'Demand': 552.0, 'Supply': 554.0},
  '2022-Q1': {'Demand': 512.0, 'Supply': 559.0},
  '2022-Q2': {'Demand': 587.0, 'Supply': 595.0},
  '2022-Q3': {'Demand': 517.0, 'Supply': 505.0},
  '2022-Q4': {'Demand': 496.0, 'Supply': 461.0},
  '2023-Q1': {'Demand': 561.0, 'Supply': 519.0},
  '2023-Q2': {'Demand': 594.0, 'Supply': 615.0},
  '2023-Q3': {'Demand': 538.0, 'Supply': 522.0},
  '2023-Q4': {'Demand': 594.0, 'Supply': 560.0},
  '2024-Q1': {'Demand': 610.0, 'Supply': 586.0},
  '2024-Q2': {'Demand': 548.0, 'Supply': 501.0},
  '2024-Q3': {'Demand': 528.0, 'Supply': 514.0},
  '2024-Q4': {'Demand': 603.0, 'Supply': 619.0}},
 'Compute Instances': {'2021-Q1': {'Demand': 463.0, 'Supply': 482.0},
  '2021-Q2': {'Demand': 398.0, 'Supply': 443.0},
  '2021-Q3': {'Demand': 364.0, 'Supply': 315.0},
  '2021-Q4': {'Demand': 418.0,

In [37]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableLambda
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import json

# Initialize the LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

def generate_price_forecast_langchain(historical_data: dict):
    """
    Uses an LLM to analyze historical supply-demand trends and predict price changes.
    
    Parameters:
        historical_data (dict): Nested dictionary containing supply-demand history.

    Returns:
        dict: Forecasted price changes per service and quarter.
    """

    # Convert historical data to JSON format
    context = json.dumps(historical_data, indent=2)

    # Adjust schema to match LLM output
    response_schemas = [
        ResponseSchema(name="forecast", description="Dictionary containing service names as keys and forecasted price changes as values.")
    ]

    # Create the output parser
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    # Define the prompt with format constraints
    prompt_template = PromptTemplate(
        input_variables=["context"],
        template="""
        You are an AI expert in **economic forecasting**.
        
        Given the **historical supply-demand data** below, analyze trends and predict **price percentage changes** 
        for the next four quarters.

        **Guidelines for Forecasting:**
        - If **demand exceeds supply**, prices **increase**.
        - If **supply exceeds demand**, prices **decrease**.
        - Consider **seasonality and demand fluctuations** over time.
        - Ensure each service has a **predicted percentage change** for the next 4 quarters.

        **Historical Data:**
        ```json
        {context}
        ```

        {format_instructions}
        """,
        partial_variables={"format_instructions": format_instructions}
    )

    # Use LangChain's new RunnableSequence API with structured parsing
    chain = prompt_template | llm | output_parser

    # Run the chain and retrieve structured output
    forecast_data = chain.invoke({"context": context})

    return forecast_data


In [41]:
def generate_negotiation_charter(price_forecast: dict):
    """
    Uses an LLM to create a detailed Negotiation Charter based on AI-generated price forecasts.

    Parameters:
        price_forecast (dict): Output from generate_price_forecast_langchain(), 
                               containing predicted price changes for services.

    Returns:
        str: A structured negotiation charter in markdown format.
    """

    # Convert price forecast data to JSON for context
    context = json.dumps(price_forecast, indent=2)

    # Define the output schema for structured response
    response_schemas = [
        ResponseSchema(name="negotiation_charter", description="A structured markdown document with price forecasts, risk analysis, and negotiation strategy."),
    ]

    # Create the output parser
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    # Define the structured prompt
    prompt_template = PromptTemplate(
        input_variables=["context"],
        template="""
        You are an **AI Procurement Strategy Expert**. 
        
        Based on the **forecasted price changes** provided below, generate a **detailed Negotiation Charter** 
        to help the procurement team **strategically negotiate better pricing, contracts, and supplier agreements**.

        **Guidelines:**
        - Analyze the **forecasted price changes** for each service.
        - Identify **high-risk suppliers** (if prices are increasing).
        - Identify **negotiation opportunities** (if prices are dropping).
        - Create a structured markdown report with the following sections:

        ---
        ## 📊 **Negotiation Charter Report**
        ### **1. Price Forecast Analysis**
        - Summarize key **price trends** for each service.
        - Highlight which services **are increasing** in price and which **are decreasing**.

        ### **2. Cost Optimization Strategy**
        - Recommendations for **cost savings** based on forecasted price movements.
        - Should the team **lock in long-term contracts, delay purchases, or renegotiate?** 

        ### **3. Supplier Risk Analysis**
        - Assess **supplier risk levels** based on **historical volatility & forecasted changes**.
        - Identify **which suppliers pose the highest cost risks**.

        ### **4. Supplier Comparison Table**
        | Supplier | Service | Current Price | Forecasted Price Change | Risk Level |
        |----------|---------|--------------|------------------------|------------|
        (Fill this table with insights)

        ### **5. Negotiation Leverage Points**
        - **Identify areas where procurement can push for better deals**.
        - **What incentives can be offered to suppliers?**
        - **Are there bulk discount opportunities?**
        - **Are there alternative suppliers with lower risk?**

        ### **6. AI-Powered Negotiation Recommendations**
        ✅ **List specific negotiation strategies** for each supplier.
        ✅ **Highlight key "asks" and "trade-offs"**.
        ✅ **Summarize the best approach for securing optimal contract terms**.

        **Forecasted Price Changes:**
        ```json
        {context}
        ```

        {format_instructions}
        """,
        partial_variables={"format_instructions": format_instructions}
    )

    # Use LangChain's new RunnableSequence API with structured parsing
    chain = prompt_template | llm | output_parser

    # Run the chain and retrieve structured output
    negotiation_charter = chain.invoke({"context": context})

    return negotiation_charter["negotiation_charter"]


# Example usage:
historical_data = load_supply_demand_forecast("supply_demand.csv")
price_forecast = generate_price_forecast_langchain(historical_data)
negotiation_charter = generate_negotiation_charter(price_forecast)

print("📄 AI-Generated Negotiation Charter:")
print(negotiation_charter)


📄 AI-Generated Negotiation Charter:
# 📊 Negotiation Charter Report

## **1. Price Forecast Analysis**
- **Cloud Storage**:  
  - **Q1**: +5.0%  
  - **Q2**: -8.0% (Opportunity to negotiate)  
  - **Q3**: +3.0%  
  - **Q4**: -2.0% (Opportunity to negotiate)
  
- **Compute Instances**:  
  - **Q1**: +7.5%  
  - **Q2**: -0.5% (Marginal decrease)
  - **Q3**: +3.0%  
  - **Q4**: -5.0% (Opportunity to negotiate)
  
- **AI-Powered Analytics**:  
  - **Q1**: +8.0%  
  - **Q2**: -3.0% (Opportunity to negotiate)
  - **Q3**: +5.0%  
  - **Q4**: -1.0% (Opportunity to negotiate)
  
- **Security & Compliance**:  
  - **Q1**: -5.0% (Opportunity to negotiate)
  - **Q2**: +4.0%  
  - **Q3**: +2.0%  
  - **Q4**: +1.0%  
  
- **Hybrid Cloud Management**:  
  - **Q1**: -1.0% (Opportunity to negotiate)
  - **Q2**: +15.0% (High-risk supplier)
  - **Q3**: +3.0%  
  - **Q4**: -5.0% (Opportunity to negotiate)

## **2. Cost Optimization Strategy**
- **Lock in long-term contracts** for Cloud Storage and AI-Power